In [2]:
!mkdir vectorizers

mkdir: cannot create directory ‘vectorizers’: File exists


In [4]:
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
#import torch
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
stopwords = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
train = pd.read_csv("data/data_generated/train_with_embeddings.csv")
test = pd.read_csv("data/data_generated/test_with_embeddings.csv")



In [18]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    #print(words)
    res = list()
    for word in words:
        if (word not in stopwords and len(word) > 1):
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    text = " ".join(res)
    return text

def text_cleaner(input_text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_url= re.sub(r'([a-z]{2}\d+[a-z]{2})',' ',rem_url)
    rem_url = re.sub(r'!+','.', rem_url)
    rem_num = re.sub('[0-9]+', '', rem_url)
    text = re.sub(r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+.,!]+", " ", rem_num)
    text = re.sub("!",".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]",'', text)
    emoji_pattern=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    text = text.lower()
    return text


def dataset_preprocessor_vectorizers(data, w_count_to_drop=3):
    data_frame = data.copy()

    indexes_to_drop = []
    # Дропаем дубли
    data_frame = data_frame.drop_duplicates(subset=['Текст инцидента']).reset_index(drop=True)

    # Regex для групп
    p = re.compile("\[.+\|.+\]")

    for index, row in tqdm(data_frame.iterrows()):

        # Очистка текста
        data_frame.loc[index, "Текст инцидента"] = text_cleaner(data_frame.loc[index, "Текст инцидента"])

        # Отсекаем сообщения из w_count_to_drop слов
        if len(row["Текст инцидента"].split(" ")) < w_count_to_drop and len(row["Текст инцидента"].split(".")) < w_count_to_drop:
            indexes_to_drop.append(index)

    # Дропаем индексы < w_count_to_drop слов
    #print(indexes_to_drop)
    print(data_frame.values.shape)
    data_frame = data_frame.drop(index=indexes_to_drop).reset_index(drop=True)

    # Лемматизация и удаление стоп слов
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = lemmatize(row["Текст инцидента"])
    return data_frame
train = dataset_preprocessor_vectorizers(train)
test = dataset_preprocessor_vectorizers(test)

0it [00:00, ?it/s]

(16850, 1028)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

(7434, 1028)


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
train = dataset_preprocessor_vectorizers(train)
test = dataset_preprocessor_vectorizers(test)

In [8]:
import pickle
class Model_tfidf:
    def __init__(self, vectorizer_name, vectorizer, max_features=None):
        self.max_features = max_features
        self.vectorizer = vectorizer(max_features=self.max_features)
        self.vectorizer_name = vectorizer_name

    def create_tfidf(self, train, test):
      tf = self.vectorizer.fit_transform(train)
      with open(f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pk', 'wb') as fin:
        pickle.dump(self.vectorizer, fin)
      vocab = self.vectorizer.vocabulary_
      train_tf = pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys()))
      test_tf = pd.DataFrame(self.vectorizer.transform(test).toarray(), columns=sorted(vocab.keys()))
      del tf
      return train_tf, test_tf


    def create_datasets(self, train, test):
        train_, test_ = train.copy(), test.copy()

        train_tf_idf, test_tf_idf = self.create_tfidf(train_["Текст инцидента"].to_list(), test_["Текст инцидента"].to_list())

        train_ = pd.concat([train_, train_tf_idf], axis=1)
        test_ = pd.concat([test_, test_tf_idf], axis=1)
        del train_tf_idf
        del test_tf_idf
        

        return train_, test_

In [9]:
mtf = Model_tfidf("tfidf", TfidfVectorizer, max_features=15000)
train_, test_  = mtf.create_datasets(train, test)

In [10]:
train_

,Исполнитель,Группа тем,Текст инцидента,Тема,0,1,2,3,4,5,...,ячневый,ящерица,ящик,ёж,ёлка,ёлки,ёлку,ёлочка,ёлочку,ёмкость
0,Город Пермь,Погребение и похоронное дело,погребение это серьёзный проблема наш страна д...,Погребение и похоронное дело,-0.294947,-0.055111,-0.917189,0.076128,-0.168686,0.458902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Город Пермь,Погребение и похоронное дело,также считать должный делать возможный поддерж...,Погребение и похоронное дело,-0.215702,0.045419,-1.404945,-0.113390,-0.199697,0.628931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Город Пермь,Погребение и похоронное дело,думать многие человек готовый переходить альте...,Погребение и похоронное дело,-0.074277,-0.069510,-1.307976,-0.731061,-0.117746,0.112123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Город Пермь,Погребение и похоронное дело,считать строительство новый крематорий стать в...,Погребение и похоронное дело,-0.215675,-0.010250,-1.629285,-0.177629,-0.034335,0.769009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Город Пермь,Погребение и похоронное дело,думать должный обсуждать вопрос открыто честно...,Погребение и похоронное дело,0.091469,-0.337995,-1.098347,-0.416570,0.188040,0.489746,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,ИГЖН ПК,ЖКХ,счёт отопление прийти месяц октябрь пол сентяб...,Завышение платы за коммунальные услуги,0.494950,-0.152662,-1.038108,-0.324845,0.389979,0.587082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16553,Министерство социального развития ПК,Социальное обслуживание и защита,здравствуйте справка получать пособие рождение...,Оказание гос. соц. помощи,-0.245126,-0.062770,-0.160016,0.977306,-0.091960,1.371442,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16554,ИГЖН ПК,ЖКХ,добрый день обращаться ненадлежащий состояние ...,Ненадлежащее содержание и эксплуатация МКД,0.096977,0.334582,0.256389,0.675268,-0.216912,0.327004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16555,ИГЖН ПК,ЖКХ,приморский управляйка чистить снег крыша вчера...,★ Наледь и сосульки на кровле,-0.198398,-0.084078,-0.520404,-1.223011,0.486935,0.555129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
train_.to_csv("data/data_generated/train_with_embeddings_tfidf.csv", index=False)
test_.to_csv("data/data_generated/test_with_embeddings_tfidf.csv", index=False)

In [13]:
param_grid = {
    'iterations': [100, 250, 400]
}
grid = list(ParameterGrid(param_grid))

In [14]:
mlflow.set_experiment('tfidf_15000features')

2023/11/25 00:14:23 INFO mlflow.tracking.fluent: Experiment with name 'tfidf_15000features' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///workspace/mlruns/632502958196186195', creation_time=1700871263925, experiment_id='632502958196186195', last_update_time=1700871263925, lifecycle_stage='active', name='tfidf_15000features', tags={}>

In [15]:

model_name, max_features = "tfidf", 15000

for params in grid:
    try:
        with mlflow.start_run(nested=True):
            catboost_model = CatBoostClassifier(**params, verbose=0, random_seed=42)
            catboost_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), train_["Группа тем"])

            predictions = catboost_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))

            accuracy = accuracy_score(test_["Группа тем"], predictions)
            f1 = f1_score(test_["Группа тем"], predictions, average='weighted')
            report = classification_report(test_["Группа тем"], predictions, output_dict=True)
            report_text =  classification_report(test_["Группа тем"], predictions)


            mlflow.log_metric("report_accuracy", report['accuracy'])
            mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
            mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
            mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
            mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
            mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
            mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])

            mlflow.log_text(report_text, "classification_report.txt")

            mlflow.set_tag("vectorizer_name", model_name)
            mlflow.set_tag("max_features", max_features)
            mlflow.set_tag("dataset_name", "train_with_embeddings")
            mlflow.set_tag("model_name", "catboost")
            mlflow.catboost.log_model(catboost_model, "model")
            mlflow.log_params(params)
            mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
    except Exception as e:
        error_name = type(e).__name__
        print(f"Caught an error: {error_name}")

In [ ]:
[]